In [147]:
from facenet_pytorch import MTCNN
import cv2
import torch
import numpy as np
import torchvision.transforms as tt
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

from Model.ResNet import MaskModel

In [148]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [149]:
mtcnn = MTCNN(keep_all=True, device=device)

In [198]:
image = cv2.imread("../Image/multiple8.jpeg")
imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [199]:
faces, _ = mtcnn.detect(imageRGB)

In [200]:
print(faces)

[[46.91210174560547 248.69467163085938 107.16472625732422
  327.5488586425781]
 [478.0069580078125 227.27606201171875 537.1964721679688
  299.21783447265625]
 [145.26490783691406 171.65464782714844 187.70606994628906
  227.37525939941406]
 [321.5096740722656 119.49620819091797 363.9490051269531
  170.91473388671875]
 [209.72344970703125 98.52619934082031 246.4689483642578
  142.97792053222656]
 [155.6968994140625 7.415914058685303 192.07400512695312
  51.974830627441406]
 [153.16053771972656 52.72328186035156 192.97012329101562
  92.74662017822266]
 [338.32769775390625 24.6259822845459 373.4423522949219 67.51487731933594]
 [497.06304931640625 69.79196166992188 531.2730712890625
  112.69622039794922]
 [28.838096618652344 122.72537994384766 63.273094177246094
  161.37203979492188]
 [263.78125 119.59635162353516 297.4248352050781 157.0006866455078]
 [49.6408805847168 42.340721130371094 80.21086120605469 83.36463928222656]
 [362.2096862792969 48.96070861816406 392.26531982421875
  87.70044

In [201]:
len(faces)

20

In [202]:
for face in faces:
    print(face)

[46.91210174560547 248.69467163085938 107.16472625732422 327.5488586425781]
[478.0069580078125 227.27606201171875 537.1964721679688 299.21783447265625]
[145.26490783691406 171.65464782714844 187.70606994628906
 227.37525939941406]
[321.5096740722656 119.49620819091797 363.9490051269531 170.91473388671875]
[209.72344970703125 98.52619934082031 246.4689483642578 142.97792053222656]
[155.6968994140625 7.415914058685303 192.07400512695312 51.974830627441406]
[153.16053771972656 52.72328186035156 192.97012329101562 92.74662017822266]
[338.32769775390625 24.6259822845459 373.4423522949219 67.51487731933594]
[497.06304931640625 69.79196166992188 531.2730712890625 112.69622039794922]
[28.838096618652344 122.72537994384766 63.273094177246094
 161.37203979492188]
[263.78125 119.59635162353516 297.4248352050781 157.0006866455078]
[49.6408805847168 42.340721130371094 80.21086120605469 83.36463928222656]
[362.2096862792969 48.96070861816406 392.26531982421875 87.70044708251953]
[378.49462890625 30.

In [203]:
model = torch.load("../kaggle/model.pt", map_location = device)

In [204]:
transform = tt.Compose([
                tt.Resize(size=(256, 256)),
                tt.CenterCrop(224),
                tt.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

In [205]:
if faces is not None:
        for face in faces:
            try:
                x1 = round(face[0])
                y1 = round(face[1])
                x2 = round(face[2])
                y2 = round(face[3])
                face_image = imageRGB[y1 : y2, x1 : x2, :]
                img = torch.tensor(face_image)
                img = img.permute(2, 0, 1)
                # image value range [0, 255] convert it to [0,1]
                img = (img-0)/(255-0)
                img = transform(img)
                img = img[None, :]
                
                model.eval()
                with torch.no_grad():
                    y = model(img.to(device))
                    y = F.softmax(y, dim = 1)
                    value, pred =torch.max(y, dim = 1)
                
                
                if pred.item() == 0:
                    color = (0, 255, 0)
                else:
                    color = (0, 0, 255)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
                classes = ['with_mask', 'without_mask']
                cv2.putText(
                            image,
                            classes[pred.item()]+f"({str(round(value.item(), 4))})",
                            (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            color,
                            1,
                            cv2.LINE_AA,
                        )
            except Exception as e:
                print(e)
else:
    cv2.putText(
            image,
            "No face found",
            (20, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 0),
            1,
            cv2.LINE_AA,
        )


In [207]:
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

live video

In [ ]:
mtcnn = MTCNN(keep_all=True, device=device)
model = torch.load("../kaggle/model.pt", map_location = device)
transform = tt.Compose([
                tt.Resize(size=(256, 256)),
                tt.CenterCrop(224),
                tt.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

In [186]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 10)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)
while True:
    ret, image = cap.read()
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces, _ = mtcnn.detect(imageRGB)
    
    if faces is not None:
        for face in faces:
            try:
                x1 = round(face[0])
                y1 = round(face[1])
                x2 = round(face[2])
                y2 = round(face[3])
                face_image = imageRGB[y1 : y2, x1 : x2, :]
                img = torch.tensor(face_image)
                img = img.permute(2, 0, 1)
                # image value range [0, 255] convert it to [0,1]
                img = (img-0)/(255-0)
                img = transform(img)
                img = img[None, :]
                model.eval()
                with torch.no_grad():
                    y = model(img.to(device))
                    y = F.softmax(y, dim = 1)
                    value, pred =torch.max(y, dim = 1)
                
                
                if pred.item() == 0:
                    color = (0, 255, 0)
                else:
                    color = (0, 0, 255)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
                classes = ['with_mask', 'without_mask']
                cv2.putText(
                            image,
                            classes[pred.item()]+f"({str(round(value.item(), 4))})",
                            (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            color,
                            1,
                            cv2.LINE_AA,
                        )
            except Exception as e:
                print(e)
    else:
        cv2.putText(
                image,
                "No face found",
                (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 0),
                1,
                cv2.LINE_AA,
            )
    
    cv2.imshow('frame', image)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
